In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split, cross_val_predict

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

#### Train Models

In [3]:
train = pd.read_csv('df_train.csv')
test = pd.read_csv('df_test.csv')

In [4]:
# Create X and y for train and test sets 
X_train = train.drop('readmitted', axis=1)
y_train = train['readmitted']

X_test = test.drop('readmitted', axis=1)
y_test = test['readmitted']

In [5]:
train.head()

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,A1Cresult_0,A1Cresult_1,diag_1_1.0,diag_1_2.0,diag_1_3.0,diag_1_4.0,diag_1_5.0,diag_1_6.0,diag_1_7.0,diag_1_8.0
0,149190,55629189,2,-0.438603,0.805968,-0.809310,0.277594,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,0
1,500364,82442376,4,-0.777582,0.051157,-0.238557,0.036471,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,0
2,16680,42519267,5,-1.116561,0.403402,-0.809310,-0.928017,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,1
3,35754,82637451,6,-0.438603,-0.603012,2.615208,0.036471,-0.264763,-0.205982,-0.329314,...,0,0,1,0,0,0,0,0,0,0
4,55842,84259809,7,-0.099625,1.359497,-0.238557,0.639277,-0.264763,-0.205982,-0.329314,...,0,0,1,0,0,0,0,0,0,0


#### Resample from train dataset to address imbalance using SMOTE

In [6]:
from imblearn.over_sampling import SMOTE
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
sm = SMOTE(random_state=42)
X_train_new, y_train_new = sm.fit_sample(X_train, y_train)
print('New dataset shape {}'.format(Counter(y_train_new)))

Original dataset shape Counter({0: 47345, 1: 4414})
New dataset shape Counter({0: 47345, 1: 47345})


In [7]:
from imblearn.under_sampling import RandomUnderSampler
print('Original dataset shape {}'.format(Counter(y_train)))
sm2 = RandomUnderSampler(random_state=42)
X_train_new2, y_train_new2 = sm2.fit_sample(X_train, y_train)
print('New dataset shape {}'.format(Counter(y_train_new2)))

Original dataset shape Counter({0: 47345, 1: 4414})
New dataset shape Counter({0: 4414, 1: 4414})


#### Train Models

In [8]:
knn = KNeighborsClassifier()
training_start = time.perf_counter()
knn.fit(X_train_new, y_train_new)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = knn.predict(X_test)
prediction_end = time.perf_counter()
acc_knn = (preds == y_test).sum().astype(float) / len(preds)*100    
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
knn_train_time = training_end-training_start
knn_prediction_time = prediction_end-prediction_start
                    
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f seconds" % (knn_train_time))
print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

Accuracy: 67.0229%
Precision: 4.8043%
Recall: 32.7549%
AUC: 0.5071579880603305
Time consumed for training: 0.603 seconds
Time consumed for prediction: 0.32690 seconds


In [9]:
knn = KNeighborsClassifier()
training_start = time.perf_counter()
knn.fit(X_train_new2, y_train_new2)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = knn.predict(X_test)
prediction_end = time.perf_counter()
acc_knn = (preds == y_test).sum().astype(float) / len(preds)*100    
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
knn_train_time = training_end-training_start
knn_prediction_time = prediction_end-prediction_start
                    
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f seconds" % (knn_train_time))
print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

Accuracy: 52.2621%
Precision: 4.9635%
Recall: 51.6269%
AUC: 0.5195980580778078
Time consumed for training: 0.085 seconds
Time consumed for prediction: 0.24271 seconds


In [10]:
xgb = XGBClassifier(n_estimators=100)
training_start = time.perf_counter()
xgb.fit(X_train_new, y_train_new)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = xgb.predict(X_test.values)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Time consumed for training: 46.059
Time consumed for prediction: 0.05457 seconds


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
xgb = XGBClassifier(n_estimators=100)
training_start = time.perf_counter()
xgb.fit(X_train_new2, y_train_new2)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = xgb.predict(X_test.values)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

Accuracy: 65.1353%
Precision: 8.0332%
Recall: 62.9067%
AUC: 0.6407480279197869
Time consumed for training: 4.538
Time consumed for prediction: 0.07250 seconds


In [12]:
gnb = GaussianNB()
training_start = time.perf_counter()
gnb.fit(X_train_new, y_train_new)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = gnb.predict(X_test)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
gnb_train_time = training_end-training_start
gnb_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (gnb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (gnb_prediction_time))

Accuracy: 30.4504%
Precision: 5.2936%
Recall: 83.5141%
AUC: 0.5570177350963059
Time consumed for training: 0.221
Time consumed for prediction: 0.01618 seconds


In [13]:
gnb = GaussianNB()
training_start = time.perf_counter()
gnb.fit(X_train_new2, y_train_new2)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = gnb.predict(X_test)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
gnb_train_time = training_end-training_start
gnb_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (gnb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (gnb_prediction_time))

Accuracy: 31.1895%
Precision: 5.2734%
Recall: 82.2126%
AUC: 0.5546977475955337
Time consumed for training: 0.014
Time consumed for prediction: 0.01735 seconds


In [14]:
rfc = RandomForestClassifier(n_estimators=10)
training_start = time.perf_counter()
rfc.fit(X_train_new, y_train_new)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = rfc.predict(X_test)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

Accuracy: 95.3161%
Precision: 25.0000%
Recall: 0.8677%
AUC: 0.5037102540904087
Time consumed for training: 3.179 seconds
Time consumed for prediction: 0.06219 seconds


In [15]:
rfc = RandomForestClassifier(n_estimators=10)
training_start = time.perf_counter()
rfc.fit(X_train_new2, y_train_new2)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = rfc.predict(X_test)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

Accuracy: 65.6447%
Precision: 6.4855%
Recall: 48.1562%
AUC: 0.5732243783995901
Time consumed for training: 0.266 seconds
Time consumed for prediction: 0.03404 seconds


In [16]:
svc = SVC()
training_start = time.perf_counter()
svc.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = svc.predict(X_test)
prediction_end = time.perf_counter()
acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
svc_train_time = training_end-training_start
svc_prediction_time = prediction_end-prediction_start

print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f seconds" % (svc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (svc_prediction_time))

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Time consumed for training: 1424.392 seconds
Time consumed for prediction: 45.92147 seconds


/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.9,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]
%time xgb.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[02:22:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[0]	validation_0-auc:0.604914	validation_0-error:0.08528	validation_1-auc:0.646365	validation_1-error:0.04604
[02:22:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 22 pruned nodes, max_depth=3
[1]	validation_0-auc:0.617977	validation_0-error:0.08528	validation_1-auc:0.655051	validation_1-error:0.04604
[02:22:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 16 pruned nodes, max_depth=3
[2]	validation_0-auc:0.62121	validation_0-error:0.08528	validation_1-auc:0.667749	validation_1-error:0.04604
[02:22:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=3
[3]	validation_0-auc:0.616406	validation_0-error:0.08528	validation_1-auc:0.661895	validation_1-error:0.04604
[02:22:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=3

[02:22:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[37]	validation_0-auc:0.63093	validation_0-error:0.08528	validation_1-auc:0.681057	validation_1-error:0.04604
[02:22:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 18 pruned nodes, max_depth=3
[38]	validation_0-auc:0.631132	validation_0-error:0.08528	validation_1-auc:0.681642	validation_1-error:0.04604
[02:22:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=3
[39]	validation_0-auc:0.631249	validation_0-error:0.08528	validation_1-auc:0.681961	validation_1-error:0.04604
[02:22:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 22 pruned nodes, max_depth=3
[40]	validation_0-auc:0.631459	validation_0-error:0.08528	validation_1-auc:0.682493	validation_1-error:0.04604
[02:22:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_dept

[02:22:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 22 pruned nodes, max_depth=3
[74]	validation_0-auc:0.632718	validation_0-error:0.08528	validation_1-auc:0.685157	validation_1-error:0.04604
[02:22:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[75]	validation_0-auc:0.632661	validation_0-error:0.08528	validation_1-auc:0.684938	validation_1-error:0.04604
[02:22:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=2
[76]	validation_0-auc:0.632602	validation_0-error:0.08528	validation_1-auc:0.684899	validation_1-error:0.04604
[02:22:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=4
[77]	validation_0-auc:0.632505	validation_0-error:0.08528	validation_1-auc:0.684965	validation_1-error:0.04604
[02:22:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 22 pruned nodes, max_dep

[02:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[111]	validation_0-auc:0.633312	validation_0-error:0.08528	validation_1-auc:0.685157	validation_1-error:0.04604
[02:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=3
[112]	validation_0-auc:0.633304	validation_0-error:0.08528	validation_1-auc:0.685324	validation_1-error:0.04604
[02:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[113]	validation_0-auc:0.633431	validation_0-error:0.08528	validation_1-auc:0.685192	validation_1-error:0.04604
[02:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=4
[114]	validation_0-auc:0.633635	validation_0-error:0.08528	validation_1-auc:0.685986	validation_1-error:0.04604
[02:22:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 20 pruned nodes, max_

[02:22:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[148]	validation_0-auc:0.635785	validation_0-error:0.08528	validation_1-auc:0.690476	validation_1-error:0.04604
[02:22:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[149]	validation_0-auc:0.635887	validation_0-error:0.08528	validation_1-auc:0.690661	validation_1-error:0.04604
[02:22:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[150]	validation_0-auc:0.635939	validation_0-error:0.08528	validation_1-auc:0.690642	validation_1-error:0.04604
[02:22:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=3
[151]	validation_0-auc:0.635935	validation_0-error:0.08528	validation_1-auc:0.69042	validation_1-error:0.04604
[02:22:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, m

...

[886]	validation_0-auc:0.664869	validation_0-error:0.08528	validation_1-auc:0.709238	validation_1-error:0.04604
[02:25:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=3
[887]	validation_0-auc:0.664887	validation_0-error:0.08528	validation_1-auc:0.70918	validation_1-error:0.04604
[02:25:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[888]	validation_0-auc:0.664887	validation_0-error:0.08528	validation_1-auc:0.70918	validation_1-error:0.04604
[02:25:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 20 pruned nodes, max_depth=4
[889]	validation_0-auc:0.664913	validation_0-error:0.08528	validation_1-auc:0.709236	validation_1-error:0.04604
[02:25:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[890]	validation_0-auc:0.664913	validation_0-error:0.08528	validation_1-auc:0.709236	validation_1-error:0.

[923]	validation_0-auc:0.66548	validation_0-error:0.08528	validation_1-auc:0.70933	validation_1-error:0.04604
[02:26:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[924]	validation_0-auc:0.66548	validation_0-error:0.08528	validation_1-auc:0.70933	validation_1-error:0.04604
[02:26:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[925]	validation_0-auc:0.66548	validation_0-error:0.08528	validation_1-auc:0.70933	validation_1-error:0.04604
[02:26:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 20 pruned nodes, max_depth=0
[926]	validation_0-auc:0.66548	validation_0-error:0.08528	validation_1-auc:0.70933	validation_1-error:0.04604
[02:26:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[927]	validation_0-auc:0.66548	validation_0-error:0.08528	validation_1-auc:0.70933	validation_1-error:0.04604
[0

[960]	validation_0-auc:0.665792	validation_0-error:0.08528	validation_1-auc:0.709388	validation_1-error:0.04604
[02:26:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[961]	validation_0-auc:0.665792	validation_0-error:0.08528	validation_1-auc:0.709388	validation_1-error:0.04604
[02:26:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[962]	validation_0-auc:0.665792	validation_0-error:0.08528	validation_1-auc:0.709388	validation_1-error:0.04604
[02:26:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[963]	validation_0-auc:0.665792	validation_0-error:0.08528	validation_1-auc:0.709388	validation_1-error:0.04604
[02:26:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=4
[964]	validation_0-auc:0.665835	validation_0-error:0.08528	validation_1-auc:0.709377	validation_1-error

[997]	validation_0-auc:0.666266	validation_0-error:0.08528	validation_1-auc:0.709372	validation_1-error:0.04604
[02:26:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[998]	validation_0-auc:0.666266	validation_0-error:0.08528	validation_1-auc:0.709372	validation_1-error:0.04604
[02:26:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=4
[999]	validation_0-auc:0.666307	validation_0-error:0.08528	validation_1-auc:0.709321	validation_1-error:0.04604
CPU times: user 4min 8s, sys: 1.19 s, total: 4min 9s
Wall time: 4min 9s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=10, learning_rate=0.01,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.3,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.9)

In [18]:
m1 = xgb.feature_importances_
m1

array([0.14402843, 0.13391638, 0.04345449, 0.01776442, 0.0204974 ,
       0.00327958, 0.01448483, 0.00218639, 0.00819896, 0.03908172,
       0.01311834, 0.01120525, 0.01503143, 0.0005466 , 0.0008199 ,
       0.        , 0.        , 0.        , 0.        , 0.00109319,
       0.0005466 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00163979, 0.0002733 ,
       0.        , 0.        , 0.        , 0.        , 0.0008199 ,
       0.021044  , 0.        , 0.00300629, 0.01530473, 0.0215906 ,
       0.00519268, 0.00300629, 0.03279585, 0.0221372 , 0.03361574,
       0.03224925, 0.02814977, 0.07461055, 0.00191309, 0.02377699,
       0.01038535, 0.        , 0.        , 0.        , 0.        ,
       0.0005466 , 0.00191309, 0.        , 0.        , 0.07105766,
       0.00765236, 0.021044  , 0.00218639, 0.        , 0.        ,
       0.        , 0.03033616, 0.0005466 , 0.0005466 , 0.        ,
       0.0008199 , 0.        , 0.        , 0.00109319, 0.00054

In [24]:
i1 = pd.DataFrame({'Variable':X_train.columns,
              'Importance': m1}).sort_values('Importance', ascending=False)
i1

,Variable,Importance
0,encounter_id,0.144028
1,patient_nbr,0.133916
47,age|number_diagnoses,0.074611
59,discharge_disposition_id_2,0.071058
2,age,0.043454
9,number_inpatient,0.039082
44,time_in_hospital|num_lab_procedures,0.033616
42,num_medications|time_in_hospital,0.032796
45,num_medications|num_lab_procedures,0.032249
66,discharge_disposition_id_28,0.030336


In [25]:
# using SMOTE X,y train set
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)
eval_set = [(X_train_new, y_train_new), (X_test.values, y_test.values)]
eval_metric = ["auc","error"]
%time xgb.fit(X_train_new, y_train_new, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[02:35:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[0]	validation_0-auc:0.868355	validation_0-error:0.168434	validation_1-auc:0.540646	validation_1-error:0.04604
[02:35:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=4
[1]	validation_0-auc:0.89547	validation_0-error:0.127775	validation_1-auc:0.536187	validation_1-error:0.04604
[02:35:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[2]	validation_0-auc:0.917169	validation_0-error:0.124997	validation_1-auc:0.54883	validation_1-error:0.04604
[02:35:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[3]	validation_0-auc:0.926362	validation_0-error:0.144524	validation_1-auc:0.555115	validation_1-error:0.04604
[02:35:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=

[02:36:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[37]	validation_0-auc:0.958687	validation_0-error:0.073809	validation_1-auc:0.611487	validation_1-error:0.04604
[02:36:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[38]	validation_0-auc:0.958865	validation_0-error:0.071708	validation_1-auc:0.610952	validation_1-error:0.04604
[02:36:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[39]	validation_0-auc:0.95882	validation_0-error:0.071549	validation_1-auc:0.611225	validation_1-error:0.04604
[02:36:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[40]	validation_0-auc:0.959002	validation_0-error:0.071053	validation_1-auc:0.612833	validation_1-error:0.04604
[02:36:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_d

[02:36:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[74]	validation_0-auc:0.960717	validation_0-error:0.0661	validation_1-auc:0.616157	validation_1-error:0.04604
[02:36:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[75]	validation_0-auc:0.960768	validation_0-error:0.065667	validation_1-auc:0.616181	validation_1-error:0.04604
[02:36:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[76]	validation_0-auc:0.960853	validation_0-error:0.065825	validation_1-auc:0.616722	validation_1-error:0.04604
[02:36:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[77]	validation_0-auc:0.960881	validation_0-error:0.065635	validation_1-auc:0.617348	validation_1-error:0.04604
[02:36:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_de

[02:36:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[111]	validation_0-auc:0.961783	validation_0-error:0.062214	validation_1-auc:0.623377	validation_1-error:0.04604
[02:36:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[112]	validation_0-auc:0.961798	validation_0-error:0.062108	validation_1-auc:0.623643	validation_1-error:0.04604
[02:36:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[113]	validation_0-auc:0.961846	validation_0-error:0.061812	validation_1-auc:0.624159	validation_1-error:0.04604
[02:36:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[114]	validation_0-auc:0.961854	validation_0-error:0.061823	validation_1-auc:0.623989	validation_1-error:0.04604
[02:36:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, 

[147]	validation_0-auc:0.962386	validation_0-error:0.060123	validation_1-auc:0.632578	validation_1-error:0.04604
[02:37:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[148]	validation_0-auc:0.962419	validation_0-error:0.060101	validation_1-auc:0.633093	validation_1-error:0.04604
[02:37:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[149]	validation_0-auc:0.962426	validation_0-error:0.059721	validation_1-auc:0.632723	validation_1-error:0.04604
[02:37:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=4
[150]	validation_0-auc:0.962428	validation_0-error:0.059658	validation_1-auc:0.632486	validation_1-error:0.04604
[02:37:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[151]	validation_0-auc:0.962444	validation_0-error:0.059542	validation_1-auc:0.632539	validation_1-e

...

[877]	validation_0-auc:0.967433	validation_0-error:0.04802	validation_1-auc:0.688492	validation_1-error:0.04604
[02:42:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=4
[878]	validation_0-auc:0.967443	validation_0-error:0.04802	validation_1-auc:0.688566	validation_1-error:0.04604
[02:42:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 20 pruned nodes, max_depth=4
[879]	validation_0-auc:0.967445	validation_0-error:0.048009	validation_1-auc:0.688508	validation_1-error:0.04604
[02:42:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=4
[880]	validation_0-auc:0.967448	validation_0-error:0.04802	validation_1-auc:0.688566	validation_1-error:0.04604
[02:42:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 12 pruned nodes, max_depth=4
[881]	validation_0-auc:0.96745	validation_0-error:0.04802	validation_1-auc:0.688596	validation_1-error

[02:43:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[914]	validation_0-auc:0.967576	validation_0-error:0.047914	validation_1-auc:0.689746	validation_1-error:0.04604
[02:43:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=4
[915]	validation_0-auc:0.967582	validation_0-error:0.047914	validation_1-auc:0.689764	validation_1-error:0.04604
[02:43:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 18 pruned nodes, max_depth=4
[916]	validation_0-auc:0.967584	validation_0-error:0.047914	validation_1-auc:0.68977	validation_1-error:0.04604
[02:43:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 8 pruned nodes, max_depth=4
[917]	validation_0-auc:0.967596	validation_0-error:0.047914	validation_1-auc:0.689846	validation_1-error:0.04604
[02:43:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes

[950]	validation_0-auc:0.967745	validation_0-error:0.04784	validation_1-auc:0.690582	validation_1-error:0.04604
[02:43:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 14 pruned nodes, max_depth=4
[951]	validation_0-auc:0.967747	validation_0-error:0.04783	validation_1-auc:0.690581	validation_1-error:0.04604
[02:43:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 16 pruned nodes, max_depth=4
[952]	validation_0-auc:0.96775	validation_0-error:0.04784	validation_1-auc:0.690515	validation_1-error:0.04604
[02:43:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 10 pruned nodes, max_depth=4
[953]	validation_0-auc:0.967751	validation_0-error:0.04784	validation_1-auc:0.690523	validation_1-error:0.04604
[02:43:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 14 pruned nodes, max_depth=4
[954]	validation_0-auc:0.967754	validation_0-error:0.04784	validation_1-auc:0.690561	validation_1-erro

[02:43:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 14 pruned nodes, max_depth=4
[987]	validation_0-auc:0.967864	validation_0-error:0.047745	validation_1-auc:0.691821	validation_1-error:0.04604
[02:43:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 14 pruned nodes, max_depth=4
[988]	validation_0-auc:0.967868	validation_0-error:0.047745	validation_1-auc:0.69186	validation_1-error:0.04604
[02:43:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=4
[989]	validation_0-auc:0.96787	validation_0-error:0.047745	validation_1-auc:0.691818	validation_1-error:0.04604
[02:43:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 16 pruned nodes, max_depth=4
[990]	validation_0-auc:0.967872	validation_0-error:0.047745	validation_1-auc:0.691825	validation_1-error:0.04604
[02:43:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 10 pruned nodes

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=10, learning_rate=0.01,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.3,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.8)

In [26]:
m2 = xgb.feature_importances_
m2

array([0.03030655, 0.01312123, 0.07210869, 0.04180214, 0.00836043,
       0.05167209, 0.00789596, 0.010915  , 0.01230841, 0.04064097,
       0.0265908 , 0.03460288, 0.02984208, 0.02380399, 0.00197399,
       0.        , 0.        , 0.00452856, 0.        , 0.01219229,
       0.01335346, 0.        , 0.00603809, 0.00197399, 0.00046447,
       0.        , 0.        , 0.        , 0.01927543, 0.00011612,
       0.        , 0.        , 0.        , 0.        , 0.0251974 ,
       0.01300511, 0.        , 0.01672085, 0.01451463, 0.02136554,
       0.00708314, 0.00917325, 0.00429633, 0.00603809, 0.00371575,
       0.00789596, 0.00859266, 0.02937761, 0.00510915, 0.00859266,
       0.00603809, 0.        , 0.02670692, 0.00011612, 0.        ,
       0.04191826, 0.01358569, 0.        , 0.00278681, 0.0606131 ,
       0.00534138, 0.00290293, 0.00336739, 0.        , 0.        ,
       0.        , 0.0088249 , 0.00673479, 0.03390618, 0.        ,
       0.00034835, 0.        , 0.00034835, 0.00092894, 0.00290

In [27]:
i2 = pd.DataFrame({'Variable':X_train.columns,
              'Importance': m2}).sort_values('Importance', ascending=False)
i2

,Variable,Importance
2,age,0.072109
59,discharge_disposition_id_2,0.060613
5,num_procedures,0.051672
76,diag_1_1.0,0.044124
55,gender_1,0.041918
3,time_in_hospital,0.041802
9,number_inpatient,0.040641
11,diag_3,0.034603
68,admission_source_id_7,0.033906
0,encounter_id,0.030307


In [28]:
# downsampled X,y train
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.9,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=4, 
                      gamma=10)
eval_set = [(X_train_new2, y_train_new2), (X_test.values, y_test.values)]
eval_metric = ["auc","error"]
%time xgb.fit(X_train_new2, y_train_new2, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[02:44:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 12 pruned nodes, max_depth=4
[0]	validation_0-auc:0.588832	validation_0-error:0.434866	validation_1-auc:0.629553	validation_1-error:0.464996
[02:44:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[1]	validation_0-auc:0.612795	validation_0-error:0.417422	validation_1-auc:0.660492	validation_1-error:0.409867
[02:44:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[2]	validation_0-auc:0.615063	validation_0-error:0.415043	validation_1-auc:0.668407	validation_1-error:0.375312
[02:44:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[3]	validation_0-auc:0.620204	validation_0-error:0.416402	validation_1-auc:0.670971	validation_1-error:0.378907
[02:44:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes,

[36]	validation_0-auc:0.653521	validation_0-error:0.393068	validation_1-auc:0.691562	validation_1-error:0.393588
[02:44:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=4
[37]	validation_0-auc:0.653637	validation_0-error:0.391821	validation_1-auc:0.690556	validation_1-error:0.394587
[02:44:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[38]	validation_0-auc:0.654377	validation_0-error:0.389783	validation_1-auc:0.691912	validation_1-error:0.390792
[02:44:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[39]	validation_0-auc:0.654088	validation_0-error:0.390122	validation_1-auc:0.69158	validation_1-error:0.390992
[02:44:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[40]	validation_0-auc:0.654264	validation_0-error:0.390122	validation_1-auc:0.691339	validation_1

[02:44:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=2
[73]	validation_0-auc:0.653401	validation_0-error:0.392048	validation_1-auc:0.691961	validation_1-error:0.39199
[02:44:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[74]	validation_0-auc:0.653416	validation_0-error:0.392728	validation_1-auc:0.692162	validation_1-error:0.391391
[02:44:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[75]	validation_0-auc:0.653347	validation_0-error:0.393634	validation_1-auc:0.692165	validation_1-error:0.39259
[02:44:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=4
[76]	validation_0-auc:0.653217	validation_0-error:0.392728	validation_1-auc:0.691819	validation_1-error:0.394787
[02:44:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes

[109]	validation_0-auc:0.654977	validation_0-error:0.390122	validation_1-auc:0.693778	validation_1-error:0.390392
[02:44:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=4
[110]	validation_0-auc:0.65508	validation_0-error:0.390009	validation_1-auc:0.693931	validation_1-error:0.390293
[02:44:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=4
[111]	validation_0-auc:0.655275	validation_0-error:0.389783	validation_1-auc:0.694385	validation_1-error:0.387496
[02:44:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[112]	validation_0-auc:0.655331	validation_0-error:0.390009	validation_1-auc:0.694543	validation_1-error:0.387496
[02:44:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[113]	validation_0-auc:0.655645	validation_0-error:0.389556	validation_1-auc:0.69468	validatio

[02:44:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 26 pruned nodes, max_depth=1
[146]	validation_0-auc:0.657434	validation_0-error:0.390122	validation_1-auc:0.695626	validation_1-error:0.378408
[02:44:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 20 pruned nodes, max_depth=3
[147]	validation_0-auc:0.65741	validation_0-error:0.390009	validation_1-auc:0.695583	validation_1-error:0.378208
[02:44:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=4
[148]	validation_0-auc:0.657427	validation_0-error:0.390009	validation_1-auc:0.695609	validation_1-error:0.376411
[02:44:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[149]	validation_0-auc:0.657422	validation_0-error:0.389783	validation_1-auc:0.695618	validation_1-error:0.37681
[02:44:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned node

...

[02:45:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 24 pruned nodes, max_depth=2
[876]	validation_0-auc:0.670953	validation_0-error:0.381853	validation_1-auc:0.706409	validation_1-error:0.344652
[02:45:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[877]	validation_0-auc:0.670953	validation_0-error:0.38174	validation_1-auc:0.706409	validation_1-error:0.344552
[02:45:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[878]	validation_0-auc:0.670953	validation_0-error:0.38174	validation_1-auc:0.706409	validation_1-error:0.344552
[02:45:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 12 pruned nodes, max_depth=0
[879]	validation_0-auc:0.670953	validation_0-error:0.38174	validation_1-auc:0.706409	validation_1-error:0.344552
[02:45:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes,

[912]	validation_0-auc:0.671174	validation_0-error:0.381627	validation_1-auc:0.706417	validation_1-error:0.344252
[02:45:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[913]	validation_0-auc:0.671174	validation_0-error:0.381627	validation_1-auc:0.706417	validation_1-error:0.344252
[02:45:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 10 pruned nodes, max_depth=4
[914]	validation_0-auc:0.67123	validation_0-error:0.381513	validation_1-auc:0.706514	validation_1-error:0.343753
[02:45:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 12 pruned nodes, max_depth=0
[915]	validation_0-auc:0.67123	validation_0-error:0.381513	validation_1-auc:0.706514	validation_1-error:0.343753
[02:45:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[916]	validation_0-auc:0.671287	validation_0-error:0.3814	validation_1-auc:0.706554	validation_1-e

[02:45:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[949]	validation_0-auc:0.671857	validation_0-error:0.379701	validation_1-auc:0.706584	validation_1-error:0.343453
[02:45:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[950]	validation_0-auc:0.671857	validation_0-error:0.379701	validation_1-auc:0.706584	validation_1-error:0.343354
[02:45:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[951]	validation_0-auc:0.671857	validation_0-error:0.379701	validation_1-auc:0.706584	validation_1-error:0.343354
[02:45:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[952]	validation_0-auc:0.671857	validation_0-error:0.379701	validation_1-auc:0.706584	validation_1-error:0.343354
[02:45:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nod

[985]	validation_0-auc:0.672274	validation_0-error:0.380381	validation_1-auc:0.70681	validation_1-error:0.343653
[02:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 16 pruned nodes, max_depth=0
[986]	validation_0-auc:0.672274	validation_0-error:0.380381	validation_1-auc:0.70681	validation_1-error:0.343653
[02:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 20 pruned nodes, max_depth=3
[987]	validation_0-auc:0.67233	validation_0-error:0.380041	validation_1-auc:0.706854	validation_1-error:0.344053
[02:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[988]	validation_0-auc:0.67233	validation_0-error:0.380041	validation_1-auc:0.706854	validation_1-error:0.343953
[02:45:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[989]	validation_0-auc:0.67233	validation_0-error:0.380041	validation_1-auc:0.706854	validation_1-er

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=10, learning_rate=0.01,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.3,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.9)

In [29]:
m3 = xgb.feature_importances_
m3

array([0.15604499, 0.12886597, 0.02811621, 0.02999063, 0.01827554,
       0.00374883, 0.01499531, 0.0004686 , 0.00937207, 0.04029991,
       0.00937207, 0.00234302, 0.01593252, 0.00421743, 0.00140581,
       0.        , 0.        , 0.        , 0.        , 0.00234302,
       0.        , 0.        , 0.00187441, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00749766, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00093721,
       0.02014995, 0.        , 0.00281162, 0.01733833, 0.02483599,
       0.00609185, 0.00093721, 0.01874414, 0.01593252, 0.03514527,
       0.03889409, 0.02436738, 0.07731958, 0.00843486, 0.01827554,
       0.0131209 , 0.        , 0.0004686 , 0.        , 0.        ,
       0.00281162, 0.00093721, 0.        , 0.0004686 , 0.08575445,
       0.0004686 , 0.0126523 , 0.        , 0.        , 0.        ,
       0.        , 0.01452671, 0.0004686 , 0.0004686 , 0.        ,
       0.00374883, 0.        , 0.        , 0.        , 0.00046

In [30]:
i3 = pd.DataFrame({'Variable':X_train.columns,
              'Importance': m3}).sort_values('Importance', ascending=False)
i3

,Variable,Importance
0,encounter_id,0.156045
1,patient_nbr,0.128866
59,discharge_disposition_id_2,0.085754
47,age|number_diagnoses,0.077320
9,number_inpatient,0.040300
45,num_medications|num_lab_procedures,0.038894
44,time_in_hospital|num_lab_procedures,0.035145
77,diag_1_2.0,0.034677
76,diag_1_1.0,0.031396
3,time_in_hospital,0.029991


#### Got better scores with second dataset with new features. Remember to drop encounter_id, patient_nbr, IsTrain columns.

In [ ]:
xgbClass = XGBClassifier(booster='gbtree', colsample_bytree=0.4, gamma=0.1, learning_rate=0.15,
       max_depth=3, min_child_weight=7,
       n_estimators=1000, n_jobs=1, 
       objective='binary:logistic',
       subsample=0.9, silent=False)
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]
%time xgbClass.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
pd.DataFrame({'Variable':X_train.columns,
              'Importance':model.feature_importances_}).sort_values('Importance', ascending=False)

In [ ]:
model = XGBClassifier()
model.fit(X_train_new, y_train_new)
pd.DataFrame({'Variable':X_train.columns,
              'Importance':model.feature_importances_}).sort_values('Importance', ascending=False)

In [ ]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

xgb = XGBClassifier(nthreads=-1)  

xgb_params = {"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
              "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight" : [ 1, 3, 5, 7 ],
              "gamma" : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
              "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
              "subsample": [ 0.3, 0.4, 0.5 , 0.7 ],
              "n_estimators": st.randint(3, 40)}
gs = RandomizedSearchCV(xgb, xgb_params, n_jobs=1)  

training_start = time.perf_counter()
gs.fit(X_train_new, y_train_new)
training_end = time.perf_counter()

prediction_start = time.perf_counter()
preds = gs.predict(X_test.values)
prediction_end = time.perf_counter()

acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
xgbCV_train_time = training_end-training_start
xgbCV_prediction_time = prediction_end-prediction_start

print(gs.best_params_)
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (xgbCV_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgbCV_prediction_time))

In [ ]:
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

# params = {  
#     "n_estimators": st.randint(3, 40),
#     "max_depth": st.randint(3, 40),
#     "learning_rate": st.uniform(0.05, 0.4),
#     "colsample_bytree": one_to_left,
#     "subsample": one_to_left,
#     "gamma": st.uniform(0, 10),
#     'reg_alpha': from_zero_positive,
#     "min_child_weight": from_zero_positive,
# }

xgb = XGBClassifier(nthreads=-1)  

xgb_params = {"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
              "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight" : [ 1, 3, 5, 7 ],
              "gamma" : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
              "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
              "subsample": [ 0.3, 0.4, 0.5 , 0.7 ],
              "n_estimators": st.randint(3, 40)}
gs = RandomizedSearchCV(xgb, xgb_params, n_jobs=1)  

training_start = time.perf_counter()
gs.fit(X_train_new2, y_train_new2)
training_end = time.perf_counter()

prediction_start = time.perf_counter()
preds = gs.predict(X_test.values)
prediction_end = time.perf_counter()

acc = accuracy_score(y_test, preds)
pre = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
AUC = roc_auc_score(y_test, preds)
xgbCV_train_time = training_end-training_start
xgbCV_prediction_time = prediction_end-prediction_start

print(gs.best_params_)
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))
print("Time consumed for training: %4.3f" % (xgbCV_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgbCV_prediction_time))